## Using a CNN for object detection

This notebook contains the simple implementation of fine tuning a mobilenet to distinguish between cars and non cars. The prediction task is in a separate notebook

In [1]:
import numpy as np
import pandas as pd
import cv2

In [8]:
from keras.models import Model
from keras.layers import Input, Lambda, Conv2D, MaxPooling2D, BatchNormalization, Dense, GlobalAveragePooling2D
from keras.layers import  Flatten, Reshape, Concatenate, Activation, Dropout
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, LambdaCallback
from keras import backend as K
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


from keras.applications.mobilenet import MobileNet
from keras.applications.vgg16  import VGG16
from keras.applications.inception_v3 import InceptionV3

### Initialise the base mobilenet network with pretrained weights
The image size is set at 128 (rather than 64 as per input data) as the minimum image size for using imagenet 
weights is 128

In [5]:
img_width, img_height = (128,128)
model = MobileNet(include_top=False, weights='imagenet', input_shape = (img_width, img_height, 3))

The mobilenet model without the final predictions layers:

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 64, 64, 32)        0         
__________

In [7]:
for layer in model.layers:
    layer.trainable = False


In [9]:
def build_model(pretrained_model):
    x = GlobalAveragePooling2D()(pretrained_model.output)
    x = Dense(1024, activation='relu')(x)
    
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    
    model_out = Model(inputs = pretrained_model.input, outputs = predictions)
    return model_out

In [10]:
model_custom = build_model(model)

In [11]:
model_custom.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 64, 64, 32)        0         
__________

In [12]:
model_custom.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [13]:
train_data_dir ='data_keras/train'
val_data_dir = 'data_keras/test'
batch_size = 32
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)

val_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(128,128),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(128,128),
    batch_size=batch_size,
    class_mode='binary')

Found 12874 images belonging to 2 classes.
Found 3444 images belonging to 2 classes.


In [14]:
def logger(epoch, logs):
    if epoch %2== 0:
        print(epoch, logs)
logging_callback = LambdaCallback(on_epoch_end=logger)

In [16]:
nb_train_samples = 12874
nb_validation_samples = 3444
epochs = 50
batch_size = 32

model_custom.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples// batch_size,
    epochs=epochs,
    shuffle=True,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size,
    callbacks=[logging_callback, ModelCheckpoint('./models/mobilenet_finetune_v2.h5', save_best_only=True),
                   TensorBoard(log_dir='./logs/mobilenet_finetune/v2' )]
)

Epoch 1/50
402/402 [==============================] - 47s 118ms/step - loss: 0.0297 - acc: 0.9899 - val_loss: 0.0280 - val_acc: 0.9904
Epoch 2/50
402/402 [==============================] - 46s 115ms/step - loss: 0.0286 - acc: 0.9906 - val_loss: 0.0179 - val_acc: 0.9927
Epoch 3/50
402/402 [==============================] - 46s 114ms/step - loss: 0.0248 - acc: 0.9911 - val_loss: 0.0266 - val_acc: 0.9901
Epoch 4/50
402/402 [==============================] - 46s 114ms/step - loss: 0.0312 - acc: 0.9895 - val_loss: 0.0119 - val_acc: 0.9945
Epoch 5/50
402/402 [==============================] - 46s 113ms/step - loss: 0.0249 - acc: 0.9921 - val_loss: 0.0392 - val_acc: 0.9871
Epoch 6/50
402/402 [==============================] - 45s 113ms/step - loss: 0.0247 - acc: 0.9914 - val_loss: 0.0203 - val_acc: 0.9933
Epoch 7/50
402/402 [==============================] - 46s 113ms/step - loss: 0.0259 - acc: 0.9917 - val_loss: 0.0114 - val_acc: 0.9956
Epoch 8/50
402/402 [==============================] - 4

In [17]:
import cv2

In [18]:
model_custom.save('./models/mobilenet_finetune_v2_50epochs.h5')